# Tornadoes Landfall Analyzser
---
This code performs a comprehensive analysis of tornado occurrences for a Kaggle competition by utilizing Google Search grounding techniques. It queries a model to assess the likelihood of future tornadoes and identifies the states most at risk. The code retrieves and displays relevant grounding data, including citations from web sources, and visualizes tornado statistics using Seaborn charts. This approach combines natural language processing with real-time data retrieval to enhance the accuracy and relevance of the analysis.

### Key APIs and Libraries
___
1. Tool.Types: Tool to support Google Search in Model

2. GenerateContentConfig : Holds all the configs when making a generate content API call

3. client.models.generate_content :  Generates content based on a query and configuration, utilizing the specified model.

4. client.chats.create() : Initializes a chat session with the specified model.

5. chat.send_message() : Sends a message to the chat model and retrieves the response.

6. Markdown(): Renders text in Markdown format for display.

7. Models: gemini-2.0-flash

### Key Functionalities:
---
**Search Grounding**: Enables the model to use real-time Google Search data to enhance the accuracy of responses. 

**Retry Mechanism**: Ensures that if grounding data is incomplete, the query is retried until valid data is obtained. 

**Footnote Generation**: Creates citations for the supported text, linking back to the original sources. 

**Data Visualization**: Utilizes Seaborn to plot charts based on tornado data, enhancing the analysis with visual representation. 



Use the API¶
Start by installing and importing the Gemini API Python SDK

In [1]:
# Uninstall packages from Kaggle base image that are not needed.
!pip uninstall -qy jupyterlab jupyterlab-lsp
# Install the google-genai SDK for this codelab.
!pip install -qU 'google-genai==1.7.0'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 5.3 MB/s eta 0:00:00


In [2]:
from google import genai
from google.genai import types

from IPython.display import Markdown, HTML, display

genai.__version__

'1.7.0'

In [3]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

client = genai.Client(api_key=GOOGLE_API_KEY)

In [4]:
# And now re-run the same query with search grounding enabled.
config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
)

def query_with_grounding():
    response = client.models.generate_content(
        model='gemini-2.0-flash',
        contents="What is likelyhood of having next tornadoes and in which state?",
        config=config_with_search,
    )
    return response.candidates[0]


rc = query_with_grounding()
Markdown(rc.content.parts[0].text)

As of April 18, 2025, here's a breakdown of the tornado risk and locations:

**Current Tornado Activity:**

*   The Plains experienced tornadoes and high winds on Thursday, April 17th.
*   A large tornado was spotted in southwest Iowa on Thursday evening.
*   A tornado watch has been issued for areas including Columbus, York, Lincoln, Omaha, and Blair in Nebraska, as well as Atlantic and Carroll in Iowa. Scattered hail (up to apple-sized) and gusts up to 70 mph are possible.

**States with potential for tornadoes:**

*   **Friday, April 18:** Western Oklahoma could see supercells developing in the evening, potentially producing a couple of tornadoes.
*   **Saturday, April 19:** Texarkana and parts of the southern Plains, including the Ozarks, are being watched for potential tornado ingredients. There is a slight risk of severe thunderstorms from west-central Texas into southern Missouri. Initial supercells could pose a risk for a tornado or two from west-central TX and north TX into southeast OK/western AR.

**Looking Ahead (Easter Weekend):**

*   The severe weather risk is expected to continue through Easter Sunday.
*   While hail and damaging winds are the main threats currently, there's a possibility of a greater tornado threat on Sunday.

**General Tornado Risk Areas ("Tornado Alley"):**

*   Texas, Oklahoma, and Kansas are key states in "Tornado Alley" and experience a high frequency of tornadoes.

**Important Note:** Severe weather and tornado forecasts can change rapidly. Stay updated with your local weather forecasts and be prepared to take shelter if a warning is issued.


In [5]:
while not rc.grounding_metadata.grounding_supports or not rc.grounding_metadata.grounding_chunks:
    # If incomplete grounding data was returned, retry.
    rc = query_with_grounding()

chunks = rc.grounding_metadata.grounding_chunks
for chunk in chunks:
    print(f'{chunk.web.title}: {chunk.web.uri}')

foxweather.com: https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAJToES7h30WL7vEy8MlJk5bvofxbFGmgPfJmWOV7d7MNoqkGBeW8MLiy3d_YrpOCeMrAfWNL2v98sHUzVhyg6gIxASjm6EPKEHWCizYMg2tJvKDeVCH7Rrk8oXoXayW54O60TmTYxEIRTsC7jrRJX3e7yL3U_D5wo1XiRh92mCxQcxt4A7Z
severeweatheroutlook.com: https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAJbzqCm1TLyoiAH83RnB3aixn0cvJP2Auk0dZLIL4hWT15fL3WHHKoJNSrZtedHJIPWWzZjNRd9Cj_Z3NIlIYSgt0VBmbwpoBmbP04Z2YQk4ZqyJXkoNscOkHQF4FVmtHSPxHz4iTTUuYs07A==
foxweather.com: https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAKwG4CXmxEicYDCmEY-E7cQdSC3HMQey3_8__CqgJOBGU_UX4QS69CT4925_oieYQBdU0h93ak3E4HrCuSPOvf4ehRlCIlH2kTv23Y5g15MvhX-EHv9de_Z7h-amsIpgTpe30v3n7GD30qyGkwCxjxp
2021training.com: https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAJHoIn4LbOdNh7nN_RfpAajcIhtxf8IDHAkG0es2324NLjDxKF4Fe96eP0h0k5U_BuH1WE0sP_Nv-g2hjoVJMyBWbVNlXauVYRcssPen_xbuFxZib4_8kyKYbS9XfVVZn2LV9dmTsfts39v_bULAQ==


In [6]:
HTML(rc.grounding_metadata.search_entry_point.rendered_content)

# Provide the Tornadoes metadata 

In [7]:
from pprint import pprint

supports = rc.grounding_metadata.grounding_supports
for support in supports:
    pprint(support.to_json_dict())

{'confidence_scores': [0.95954764],
 'grounding_chunk_indices': [0],
 'segment': {'end_index': 255,
             'start_index': 185,
             'text': '*   A large tornado was spotted in southwest Iowa on '
                     'Thursday evening.'}}
{'confidence_scores': [0.9492542, 0.6671038],
 'grounding_chunk_indices': [0, 1],
 'segment': {'end_index': 658,
             'start_index': 523,
             'text': '*   **Friday, April 18:** Western Oklahoma could see '
                     'supercells developing in the evening, potentially '
                     'producing a couple of tornadoes.'}}
{'confidence_scores': [0.8240885],
 'grounding_chunk_indices': [0],
 'segment': {'end_index': 805,
             'start_index': 659,
             'text': '*   **Saturday, April 19:** Texarkana and parts of the '
                     'southern Plains, including the Ozarks, are being watched '
                     'for potential tornado ingredients.'}}
{'confidence_scores': [0.9366438],
 'gro

# Tornadoes Footnotes

In [8]:
import io

markdown_buffer = io.StringIO()

# Print the text with footnote markers.
markdown_buffer.write("Supported text:\n\n")
for support in supports:
    markdown_buffer.write(" * ")
    markdown_buffer.write(
        rc.content.parts[0].text[support.segment.start_index : support.segment.end_index]
    )

    for i in support.grounding_chunk_indices:
        chunk = chunks[i].web
        markdown_buffer.write(f"<sup>[{i+1}]</sup>")

    markdown_buffer.write("\n\n")


# And print the footnotes.
markdown_buffer.write("Citations:\n\n")
for i, chunk in enumerate(chunks, start=1):
    markdown_buffer.write(f"{i}. [{chunk.web.title}]({chunk.web.uri})\n")


Markdown(markdown_buffer.getvalue())

Supported text:

 * *   A large tornado was spotted in southwest Iowa on Thursday evening.<sup>[1]</sup>

 * *   **Friday, April 18:** Western Oklahoma could see supercells developing in the evening, potentially producing a couple of tornadoes.<sup>[1]</sup><sup>[2]</sup>

 * *   **Saturday, April 19:** Texarkana and parts of the southern Plains, including the Ozarks, are being watched for potential tornado ingredients.<sup>[1]</sup>

 * There is a slight risk of severe thunderstorms from west-central Texas into southern Missouri.<sup>[2]</sup>

 * Initial supercells could pose a risk for a tornado or two from west-central TX and north TX into southeast OK/western AR.<sup>[2]</sup>

 * *   The severe weather risk is expected to continue through Easter Sunday.<sup>[1]</sup><sup>[3]</sup>

 * *   While hail and damaging winds are the main threats currently, there's a possibility of a greater tornado threat on Sunday.<sup>[3]</sup>

 * *   Texas, Oklahoma, and Kansas are key states in "Tornado Alley" and experience a high frequency of tornadoes.<sup>[4]</sup>

Citations:

1. [foxweather.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAJToES7h30WL7vEy8MlJk5bvofxbFGmgPfJmWOV7d7MNoqkGBeW8MLiy3d_YrpOCeMrAfWNL2v98sHUzVhyg6gIxASjm6EPKEHWCizYMg2tJvKDeVCH7Rrk8oXoXayW54O60TmTYxEIRTsC7jrRJX3e7yL3U_D5wo1XiRh92mCxQcxt4A7Z)
2. [severeweatheroutlook.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAJbzqCm1TLyoiAH83RnB3aixn0cvJP2Auk0dZLIL4hWT15fL3WHHKoJNSrZtedHJIPWWzZjNRd9Cj_Z3NIlIYSgt0VBmbwpoBmbP04Z2YQk4ZqyJXkoNscOkHQF4FVmtHSPxHz4iTTUuYs07A==)
3. [foxweather.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAKwG4CXmxEicYDCmEY-E7cQdSC3HMQey3_8__CqgJOBGU_UX4QS69CT4925_oieYQBdU0h93ak3E4HrCuSPOvf4ehRlCIlH2kTv23Y5g15MvhX-EHv9de_Z7h-amsIpgTpe30v3n7GD30qyGkwCxjxp)
4. [2021training.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAJHoIn4LbOdNh7nN_RfpAajcIhtxf8IDHAkG0es2324NLjDxKF4Fe96eP0h0k5U_BuH1WE0sP_Nv-g2hjoVJMyBWbVNlXauVYRcssPen_xbuFxZib4_8kyKYbS9XfVVZn2LV9dmTsfts39v_bULAQ==)


In [9]:
from IPython.display import display, Image, Markdown

def show_response(response):
    for p in response.candidates[0].content.parts:
        if p.text:
            display(Markdown(p.text))
        elif p.inline_data:
            display(Image(p.inline_data.data))
        else:
            print(p.to_json_dict())
    
        display(Markdown('----'))

# Tornadoes Stats for the year 2024

In [10]:
config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
    temperature=0.0,
)

chat = client.chats.create(model='gemini-2.0-flash')

response = chat.send_message(
    message="What were the tornadoes tallies, by top-10 state,  from Jan 2024 to Dec 2024?",
    config=config_with_search,
)

show_response(response)

Here are the top 10 states with the highest tornado counts in 2024:

1.  **Texas:** 169
2.  **Nebraska:** 131
3.  **Iowa:** 131
4.  **Illinois:** 126
5.  **Missouri:** 105
6.  **Florida:** 103
7.  **Louisiana:** 92
8.  **Oklahoma:** 91
9.  **Kansas:** 89
10. **Ohio:** 74

It's worth noting that these numbers are preliminary estimates as of January 2025, and the final counts may vary slightly. Also, 2024 was a record-breaking year for tornadoes in several states, including Illinois, Iowa, New York, Ohio, Oklahoma, and West Virginia.


----

# Seborn Chart (State vs Tornadoes)

In [11]:
config_with_code = types.GenerateContentConfig(
    tools=[types.Tool(code_execution=types.ToolCodeExecution())],
    temperature=0.0,
)

response = chat.send_message(
    message="Plot seaborn chart with states with highest tornadoes, change color to ornage",
    config=config_with_code,
)

show_response(response)

Okay, I can help you with that. I'll generate a Python script that uses Seaborn to create a bar chart of the top 10 states with the highest tornado counts in 2024, with the bars colored orange.



----

{'executable_code': {'code': "import matplotlib.pyplot as plt\nimport seaborn as sns\n\n# Tornado data for 2024 (as provided earlier)\ntornado_data = {\n    'Texas': 169,\n    'Nebraska': 131,\n    'Iowa': 131,\n    'Illinois': 126,\n    'Missouri': 105,\n    'Florida': 103,\n    'Louisiana': 92,\n    'Oklahoma': 91,\n    'Kansas': 89,\n    'Ohio': 74\n}\n\n# Convert the dictionary to a Pandas DataFrame for easier plotting\nimport pandas as pd\ndf = pd.DataFrame(list(tornado_data.items()), columns=['State', 'Tornado Count'])\n\n# Sort the DataFrame by tornado count in descending order\ndf = df.sort_values(by='Tornado Count', ascending=False)\n\n# Create the bar chart using Seaborn\nplt.figure(figsize=(12, 6))  # Adjust figure size for better readability\nsns.barplot(x='State', y='Tornado Count', data=df, color='orange')\nplt.title('Top 10 States with Highest Tornado Counts in 2024')\nplt.xlabel('State')\nplt.ylabel('Tornado Count')\nplt.xticks(rotation=45, ha='right')  # Rotate x-axis 

----

I have generated a Python script that creates a Seaborn bar chart visualizing the top 10 states with the highest tornado counts in 2024. The bars are colored orange, and the x-axis labels are rotated for better readability. The chart displays the state names on the x-axis and the corresponding tornado counts on the y-axis.


----